In [85]:
import pandas as pd
import glob
import numpy as np
import os

In [86]:
participant_file = 'ordering2_raw.iqdat'
df = pd.read_table(participant_file, sep="\t")
participant_number = df['subject'].iloc[0]
ordering = 2 if participant_number % 2 == 0 else 1
df.head()

,date,time,group,subject,build,blocknum,trialnum,blockcode,trialcode,pretrialpause,...,stimulusonset1,stimulusnumber2,stimulusitem2,stimulusonset2,stimulusnumber3,stimulusitem3,stimulusonset3,stimulusnumber4,stimulusitem4,stimulusonset4
0,83016,12:33:44,1,4,5.0.5.0,2,1,training_block,training_trial,0,...,-8,1,dot,-8,1,rectangle,-8,1,rectangle,-8
1,83016,12:33:44,1,4,5.0.5.0,2,3,training_block,training_trial,0,...,-8,1,dot,-8,1,rectangle,-8,1,rectangle,-8
2,83016,12:33:44,1,4,5.0.5.0,2,5,training_block,training_trial,0,...,-8,1,dot,-8,1,rectangle,-8,1,rectangle,-8
3,83016,12:33:44,1,4,5.0.5.0,2,8,training_block,training_trial,0,...,-8,1,dot,-8,1,rectangle,-8,1,rectangle,-8
4,83016,12:33:44,1,4,5.0.5.0,2,11,training_block,training_trial,0,...,-8,1,dot,-8,1,rectangle,-8,1,rectangle,-8


In [87]:
columns_needed = ['date','time','subject', 'trialnum', 'blockcode', 'response', 'latency']

df = df[columns_needed]

df.head()

,date,time,subject,trialnum,blockcode,response,latency
0,83016,12:33:44,4,1,training_block,0,484
1,83016,12:33:44,4,3,training_block,0,484
2,83016,12:33:44,4,5,training_block,44,461
3,83016,12:33:44,4,8,training_block,44,335
4,83016,12:33:44,4,11,training_block,44,250


In [88]:
# Replace no responses which are represented with 0 as NaN
df = df.replace(0, np.nan)

# If there is no response, latency should be NaN, not the trial length
df.loc[df['response'].isnull(), 'latency'] = np.nan

# 30 represents the letter 'A'. 31 represents the letter 'B'. Map it use values 0 and 1 for A and B
df['response'] = df['response'].replace(
    {2: 1, 11: 0, 44:'z', 50:'m', 57: "SPACE"})

df.head()

,date,time,subject,trialnum,blockcode,response,latency
0,83016,12:33:44,4,1,training_block,NaN,NaN
1,83016,12:33:44,4,3,training_block,NaN,NaN
2,83016,12:33:44,4,5,training_block,z,461.0
3,83016,12:33:44,4,8,training_block,z,335.0
4,83016,12:33:44,4,11,training_block,z,250.0


In [89]:
block_types = df['blockcode'].drop_duplicates().tolist()
block_types

['training_block', 'practice_block', 'test_block']

In [90]:
if not os.path.exists("block_outputs"):
    os.mkdir("block_outputs")
    
if not os.path.exists("block_outputs/%d" % participant_number):
    os.mkdir("block_outputs/%d" % participant_number)

for block in block_types:
    
    print block
    
    # Only get the rows for the current block.
    block_df = df.loc[df['blockcode'] == block]
    
    # Blockcode column is not needed anymore
    block_df = block_df.drop("blockcode", axis=1)

    # Change the column name to the appropriate name
    block_df = block_df.rename(columns={'latency': 'response_time'})
    
    if block == 'test_block':
        # Set the correct block number
        block_df['stim_response'] = block_df['response'].replace({1:'Left', 0:'Right'})

    # Set the correct trial numbers
    block_df["trialnum"] = [j+1 for j in range(block_df['trialnum'].size)]

    block_df.to_csv('block_outputs/%d/%s.csv' % (participant_number, block), index=False)

training_block
practice_block
test_block


### Add stimuli for Test Block

In [91]:
stim_df = pd.read_csv('block_outputs/participant_stim/participant_%d_test.csv' % ordering)

df = pd.read_csv('block_outputs/%d/test_block.csv' % participant_number)

stim_cols = ['stim1_dots1', 'stim1_dots2', 'stim1_dots3', 'stim1_color1', 'stim1_color2', 'stim1_color3',
 'stim2_dots1', 'stim2_dots2', 'stim2_dots3', 'stim2_color1', 'stim2_color2', 'stim2_color3', 'comparison', 'test',
 'correct', 'participant']

df = df.join(stim_df[stim_cols])

df = df.rename(columns={'participant':'ordering'})

df.to_csv('block_outputs/%d/test_block.csv' % participant_number, index=False)

df.head()

,date,time,subject,trialnum,response,response_time,stim_response,stim1_dots1,stim1_dots2,stim1_dots3,...,stim2_dots1,stim2_dots2,stim2_dots3,stim2_color1,stim2_color2,stim2_color3,comparison,test,correct,ordering
0,83016,12:33:44,4,1,1.0,1088.0,Left,9,7,4,...,1,8,6,red,green,purple,1,dots,Right,2
1,83016,12:33:44,4,2,0.0,2503.0,Right,3,9,2,...,3,9,2,yellow,orange,darkgrey,7,colours,Left,2
2,83016,12:33:44,4,3,0.0,1764.0,Right,6,9,8,...,6,9,8,purple,orange,darkgrey,12,colours,Right,2


## Add stimuli for Training Block

In [92]:
stim_df = pd.read_csv('block_outputs/participant_stim/participant_%d_training.csv' % ordering)

df = pd.read_csv('block_outputs/%d/training_block.csv' % participant_number)

stim_cols = ['dot_values', 'colour_values', 'participant']

df = df.join(stim_df[stim_cols])

df = df.rename(columns={'participant':'ordering'})

df.to_csv('block_outputs/%d/training_block.csv' % participant_number, index=False)

df.head()

,date,time,subject,trialnum,response,response_time,dot_values,colour_values,ordering
0,83016,12:33:44,4,1,NaN,NaN,NaN,Yellow,2
1,83016,12:33:44,4,2,NaN,NaN,9.0,LightPink,2
2,83016,12:33:44,4,3,z,461.0,2.0,Brown,2
3,83016,12:33:44,4,4,z,335.0,4.0,Blue,2
4,83016,12:33:44,4,5,z,250.0,1.0,Red,2
